In [2]:
# dbName = "app_member"https://cintell.foresight.healtheintent.com/user/ak055384/notebooks/Users/ak055384/Document%20References/document-explore.ipynb#

# import foresight.database_util
# # Using default version will get latest version for each table, since medicationdispenses is not same version as other tables
# foresight.database_util.use_database(spark, dbName)

# from foresight.discern import with_discern_context
%matplotlib inline
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import functions as F
from pyspark.sql.functions import concat, col, lit
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils import  shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
pd.options.display.min_rows = 100
pd.options.display.max_rows = 500
pd.options.display.max_colwidth = 5000
import pandas.util.testing as tm

In [3]:
import pickle
from sklearn.manifold import TSNE
import csv
#From Scikit Learn
from sklearn import preprocessing
from sklearn.model_selection  import train_test_split, cross_val_score, KFold
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report
import time
import re
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.preprocessing import LabelEncoder
from scipy import sparse
from functools import partial

In [4]:
spark = SparkSession.builder \
                    .config('spark.dynamicAllocation.maxExecutors', '400') \
                    .config('spark.executor.cores', '5') \
                    .config('spark.sql.shuffle.partitions', '2000') \
                    .config('spark.driver.maxResultSize','40g') \
                    .getOrCreate()

In [4]:
# Location of the discernontology database
from foresight.discern import broadcast_discern, push_discern, pop_discern
root = 's3://cintell-datalab-persistence-s3-data/discernontology/v1/'

contextid = ['A092E06F76904ACEA678885913A35D46']

#Loop that removes all hypthens and captializes all letters. Setups up file location
for contextid in contextid:
    contextid = contextid.upper().replace("-", "")
    push_discern(spark, contextid, discern_root=root)
    
    print(contextid)

A092E06F76904ACEA678885913A35D46


In [5]:
!pip install mlxtend  

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [6]:
spark.sql('use intermountain_member')
spark.sql('show tables').toPandas()

,database,tableName,isTemporary
0,intermountain_member,advancedirectives,False
1,intermountain_member,allergies,False
2,intermountain_member,appointments,False
3,intermountain_member,calculatedriskscores,False
4,intermountain_member,careplans,False
5,intermountain_member,claims,False
6,intermountain_member,cmsdrgs,False
7,intermountain_member,cmsmembersinfo,False
8,intermountain_member,communications,False
9,intermountain_member,conditions,False


In [7]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [8]:
# Show database being used.
spark.sql("SELECT CURRENT_DATABASE()").toPandas()

,current_database()
0,intermountain_member


In [9]:
import foresight.database_util

dbName = "intermountain_member"
versionDF = spark.sql(F"""select max(version) as versionNumber from {dbName}.encounters order by 1 """).toPandas()
versionNumber =  versionDF.versionNumber[0] 
versionNumber = str(versionNumber) + ".0"
versionNumber = versionNumber.replace(' ', 'T')
print(versionNumber)

foresight.database_util.use_database(spark, dbName, versionNumber)

2020-05-31T20:10:00.0


In [10]:
versionDF = spark.sql(F"""select max(version) as versionNumber from {dbName}.documentreferences order by 1 """).toPandas()
versionNumber =  versionDF.versionNumber[0] 
versionNumber = str(versionNumber) + ".0"
versionNumber = versionNumber.replace(' ', 'T')
print(versionNumber)

foresight.database_util.use_database(spark, dbName, versionNumber)

2020-05-31T20:10:00.0


In [11]:
defaultcontext = 'A092E06F76904ACEA678885913A35D46'
defaultcontext

'A092E06F76904ACEA678885913A35D46'

In [12]:
DocumentTypes = """ADMINISTRATIVE_NOTE_OBSTYPE', 'BONE_DENSITY_REPORT_OBSTYPE', 'CARDIAC_CATHETERIZATION_REPORT_OBSTYPE', 'CARDIOLOGY_REPORT_OBSTYPE', 'CARE_MANAGER_NOTE_OBSTYPE', 'CLINICAL_NOTE_OBSTYPE','CLINICAL_NOTE_IP_OP_OBSTYPE', 'CONSULTATION_NOTE_OBSTYPE', 'CT_REPORT_OBSTYPE','CYTOLOGY_REPORT_OBSTYPE', 'DIAGNOSTIC_IMAGING_REPORT_OBSTYPE', 'DISCHARGE_TRANSFER_SUMMARY_NOTE_OBSTYPE','ECG_REPORT_OBSTYPE','ECHO_REPORT_OBSTYPE','ELECTRON_MICROSCOPY_REPORT_OBSTYPE','ENDOSCOPY_REPORT_OBSTYPE','FLUOROSCOPY_REPORT_OBSTYPE','H_AND_P_NOTE_OBSTYPE','HEMATOLOGY_REPORT_OBSTYPE','IMMUNIZATION_RECORD_OBSTYPE','IMMUNIZATION_SUMMARY_REPORT_OBSTYPE','IR_REPORT_OBSTYPE','MAMMOGRAPHY_REPORT_OBSTYPE','MEDICATION_MEDICAL_EQUIPMENT_MANAGEMENT_NOTE_OBSTYPE','MICROBIOLOGY_REPORT_OBSTYPE', 'MICROBIOLOGY_REPORT_DOCUMENT_OBSTYPE', 'MR_REPORT_OBSTYPE','NM_NUCLEAR_MEDICINE_REPORT_OBSTYPE','NURSE_NOTE_OBSTYPE','PATHOLOGY_REPORT_OBSTYPE','PATHOLOGY_HISTOLOGY_REPORT_OBSTYPE','PET_SCAN_REPORT_OBSTYPE', 'PHONE_ENCOUNTER_NOTE_OBSTYPE','PLAN_OF_CARE_ACTION_PLAN_NOTE_OBSTYPE','PROCEDURE_SURGERY_NOTE_OBSTYPE','RADIOLOGY_REPORT_OBSTYPE','REFERRAL_NOTE_OBSTYPE','RISK_ASSESSMENT_AND_SCREENING_NOTE_OBSTYPE','SUMMARY_DOCUMENT_OBSTYPE','THERAPY_NOTE_OBSTYPE','US_ULTRASOUND_REPORT_OBSTYPE','VA_C_P_DOCUMENT_OBSTYPE','XR_REPORT_OBSTYPE"""
DocumentTypes

"ADMINISTRATIVE_NOTE_OBSTYPE', 'BONE_DENSITY_REPORT_OBSTYPE', 'CARDIAC_CATHETERIZATION_REPORT_OBSTYPE', 'CARDIOLOGY_REPORT_OBSTYPE', 'CARE_MANAGER_NOTE_OBSTYPE', 'CLINICAL_NOTE_OBSTYPE','CLINICAL_NOTE_IP_OP_OBSTYPE', 'CONSULTATION_NOTE_OBSTYPE', 'CT_REPORT_OBSTYPE','CYTOLOGY_REPORT_OBSTYPE', 'DIAGNOSTIC_IMAGING_REPORT_OBSTYPE', 'DISCHARGE_TRANSFER_SUMMARY_NOTE_OBSTYPE','ECG_REPORT_OBSTYPE','ECHO_REPORT_OBSTYPE','ELECTRON_MICROSCOPY_REPORT_OBSTYPE','ENDOSCOPY_REPORT_OBSTYPE','FLUOROSCOPY_REPORT_OBSTYPE','H_AND_P_NOTE_OBSTYPE','HEMATOLOGY_REPORT_OBSTYPE','IMMUNIZATION_RECORD_OBSTYPE','IMMUNIZATION_SUMMARY_REPORT_OBSTYPE','IR_REPORT_OBSTYPE','MAMMOGRAPHY_REPORT_OBSTYPE','MEDICATION_MEDICAL_EQUIPMENT_MANAGEMENT_NOTE_OBSTYPE','MICROBIOLOGY_REPORT_OBSTYPE', 'MICROBIOLOGY_REPORT_DOCUMENT_OBSTYPE', 'MR_REPORT_OBSTYPE','NM_NUCLEAR_MEDICINE_REPORT_OBSTYPE','NURSE_NOTE_OBSTYPE','PATHOLOGY_REPORT_OBSTYPE','PATHOLOGY_HISTOLOGY_REPORT_OBSTYPE','PET_SCAN_REPORT_OBSTYPE', 'PHONE_ENCOUNTER_NOTE_OBSTYPE

In [13]:
documents_concat =  spark.sql(F"""select dr.encounterId,
                              collect_set(date_format(dr.serviceDate, 'yyyy-MM-dd')) as DocServiceDate,
                              collect_set(e.classification.standard.primaryDisplay) as EncounterType,
                              collect_set(e.admissionSource.standard.primaryDisplay) as EncounterAdmissionSource,
                              collect_set(e.admissionType.standard.primaryDisplay) as EncounterAdmissionType,
                              collect_set(locations[0].facility.name) as LocationFacility, 
                              collect_set(locations[0].building.name) as LocationBuilding, 
                              collect_set(locations[0].nurseUnit.name) as LocationNurseUnit,
                              collect_set(date_format(e.serviceDate, 'yyyy-MM-dd')) AS EncounterRegistrationDate,
                              collect_set(date_format(e.dischargeDate, 'yyyy-MM-dd')) AS EncounterDischargeDate,
                              collect_set(CASE WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('ADMINISTRATIVE_NOTE_OBSTYPE'), '{defaultcontext}' ) THEN 'ADMINISTRATIVE_NOTE_OBSTYPE' 
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('BONE_DENSITY_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'BONE_DENSITY_REPORT_OBSTYPE' 
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('CARDIAC_CATHETERIZATION_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'CARDIAC_CATHETERIZATION_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('CARE_MANAGER_NOTE_OBSTYPE'), '{defaultcontext}' ) THEN 'CARE_MANAGER_NOTE_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('CARDIOLOGY_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'CARDIOLOGY_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('CLINICAL_NOTE_OBSTYPE'), '{defaultcontext}' ) THEN 'CLINICAL_NOTE_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('CLINICAL_NOTE_IP_OP_OBSTYPE'), '{defaultcontext}' ) THEN 'CLINICAL_NOTE_IP_OP_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('CONSULTATION_NOTE_OBSTYPE'), '{defaultcontext}' ) THEN 'CONSULTATION_NOTE_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('CT_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'CT_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('CYTOLOGY_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'CYTOLOGY_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('DIAGNOSTIC_IMAGING_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'DIAGNOSTIC_IMAGING_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('DISCHARGE_TRANSFER_SUMMARY_NOTE_OBSTYPE'), '{defaultcontext}' ) THEN 'DISCHARGE_TRANSFER_SUMMARY_NOTE_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('ECG_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'ECG_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('ECHO_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'ECHO_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('ELECTRON_MICROSCOPY_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'ELECTRON_MICROSCOPY_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('ENDOSCOPY_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'ENDOSCOPY_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('FLUOROSCOPY_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'FLUOROSCOPY_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('H_AND_P_NOTE_OBSTYPE'), '{defaultcontext}' ) THEN 'H_AND_P_NOTE_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('HEMATOLOGY_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'HEMATOLOGY_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('IMMUNIZATION_RECORD_OBSTYPE'), '{defaultcontext}' ) THEN 'IMMUNIZATION_RECORD_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('IMMUNIZATION_SUMMARY_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'IMMUNIZATION_SUMMARY_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('IR_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'IR_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('MAMMOGRAPHY_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'MAMMOGRAPHY_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('MEDICATION_MEDICAL_EQUIPMENT_MANAGEMENT_NOTE_OBSTYPE'), '{defaultcontext}' ) THEN 'MEDICATION_MEDICAL_EQUIPMENT_MANAGEMENT_NOTE_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('MICROBIOLOGY_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'MICROBIOLOGY_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('MICROBIOLOGY_REPORT_DOCUMENT_OBSTYPE'), '{defaultcontext}' ) THEN 'MICROBIOLOGY_REPORT_DOCUMENT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('MR_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'MR_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('NM_NUCLEAR_MEDICINE_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'NM_NUCLEAR_MEDICINE_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('NURSE_NOTE_OBSTYPE'), '{defaultcontext}' ) THEN 'NURSE_NOTE_OBSTYPE' 
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('PATHOLOGY_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'PATHOLOGY_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('PATHOLOGY_HISTOLOGY_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'PATHOLOGY_HISTOLOGY_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('PET_SCAN_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'PET_SCAN_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('PHONE_ENCOUNTER_NOTE_OBSTYPE'), '{defaultcontext}' ) THEN 'PHONE_ENCOUNTER_NOTE_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('PLAN_OF_CARE_ACTION_PLAN_NOTE_OBSTYPE'), '{defaultcontext}' ) THEN 'PLAN_OF_CARE_ACTION_PLAN_NOTE_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('PROCEDURE_SURGERY_NOTE_OBSTYPE'), '{defaultcontext}' ) THEN 'PROCEDURE_SURGERY_NOTE_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('RADIOLOGY_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'RADIOLOGY_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('REFERRAL_NOTE_OBSTYPE'), '{defaultcontext}' ) THEN 'REFERRAL_NOTE_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('RISK_ASSESSMENT_AND_SCREENING_NOTE_OBSTYPE'), '{defaultcontext}' ) THEN 'RISK_ASSESSMENT_AND_SCREENING_NOTE_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('SUMMARY_DOCUMENT_OBSTYPE'), '{defaultcontext}' ) THEN 'SUMMARY_DOCUMENT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('THERAPY_NOTE_OBSTYPE'), '{defaultcontext}' ) THEN 'THERAPY_NOTE_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('US_ULTRASOUND_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'US_ULTRASOUND_REPORT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('VA_C_P_DOCUMENT_OBSTYPE'), '{defaultcontext}' ) THEN 'VA_C_P_DOCUMENT_OBSTYPE'
                                   WHEN HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('XR_REPORT_OBSTYPE'), '{defaultcontext}' ) THEN 'XR_REPORT_OBSTYPE' 
                                   ELSE NULL
                              END) AS DiscernDocumentType,
                              collect_set(dr.type.standard.primaryDisplay) AS StandardDocumentType
                          FROM {dbName}_documentreferences dr
                        JOIN {dbName}_encounters e ON dr.encounterId = e.encounterId
                          WHERE HAS_ANY_CONCEPT_IN_CONTEXT(dr.type, array('{DocumentTypes}'),'{defaultcontext}')
                          and e.classification.standard.primaryDisplay IS NOT NULL
                          and e.dischargeDate IS NOT NULL
                          and e.admissionType.standard.primaryDisplay IS NOT NULL
                          and locations[0].nurseUnit.name IS NOT NULL
                          and e.classification.standard.primaryDisplay in ('Outpatient', 'Emergency', 'Preadmit')
                          GROUP BY dr.encounterId
                          HAVING size(StandardDocumentType) <= 4
                          limit 1000000""").toPandas()
documents_concat.head()

,encounterId,DocServiceDate,EncounterType,EncounterAdmissionSource,EncounterAdmissionType,LocationFacility,LocationBuilding,LocationNurseUnit,EncounterRegistrationDate,EncounterDischargeDate,DiscernDocumentType,StandardDocumentType
0,25833853,[2015-02-24],[Outpatient],[Clinic referral],[Elective],[LG_Logan],[LG_Med Art Bldg],[LG_Budge Lab],[2015-02-24],[2015-02-25],[ADMINISTRATIVE_NOTE_OBSTYPE],[Consent Document]
1,25857613,[2015-03-01],[Emergency],[Information not available],[Emergency],[LG_Logan],[LG_Logan],[LG_ED],[2015-03-01],[2015-03-01],"[CLINICAL_NOTE_OBSTYPE, ADMINISTRATIVE_NOTE_OBSTYPE]","[Consent Document, Emergency department Note, Emergency department Triage note]"
2,25858962,[2015-03-02],[Emergency],[Information not available],[Emergency],[LG_Logan],[LG_Logan],[LG_ED],[2015-03-02],[2015-03-02],"[CLINICAL_NOTE_OBSTYPE, ADMINISTRATIVE_NOTE_OBSTYPE]","[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
3,25873399,[2015-03-02],[Outpatient],[Information not available],[Unknown],[LG_Logan],[LG_Logan],[LG_Main Lab],[2015-03-02],[2015-03-03],[PATHOLOGY_REPORT_OBSTYPE],[Surgical pathology study]
4,26095725,[2015-04-14],[Outpatient],[Clinic referral],[Elective],[LG_Logan],[LG_Logan],[LG_US],[2015-04-14],[2015-04-15],[ADMINISTRATIVE_NOTE_OBSTYPE],[Consent Document]


In [14]:
documents_concat.shape

(1000000, 12)

In [15]:
documents_concat.head()

,encounterId,DocServiceDate,EncounterType,EncounterAdmissionSource,EncounterAdmissionType,LocationFacility,LocationBuilding,LocationNurseUnit,EncounterRegistrationDate,EncounterDischargeDate,DiscernDocumentType,StandardDocumentType
0,25833853,[2015-02-24],[Outpatient],[Clinic referral],[Elective],[LG_Logan],[LG_Med Art Bldg],[LG_Budge Lab],[2015-02-24],[2015-02-25],[ADMINISTRATIVE_NOTE_OBSTYPE],[Consent Document]
1,25857613,[2015-03-01],[Emergency],[Information not available],[Emergency],[LG_Logan],[LG_Logan],[LG_ED],[2015-03-01],[2015-03-01],"[CLINICAL_NOTE_OBSTYPE, ADMINISTRATIVE_NOTE_OBSTYPE]","[Consent Document, Emergency department Note, Emergency department Triage note]"
2,25858962,[2015-03-02],[Emergency],[Information not available],[Emergency],[LG_Logan],[LG_Logan],[LG_ED],[2015-03-02],[2015-03-02],"[CLINICAL_NOTE_OBSTYPE, ADMINISTRATIVE_NOTE_OBSTYPE]","[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
3,25873399,[2015-03-02],[Outpatient],[Information not available],[Unknown],[LG_Logan],[LG_Logan],[LG_Main Lab],[2015-03-02],[2015-03-03],[PATHOLOGY_REPORT_OBSTYPE],[Surgical pathology study]
4,26095725,[2015-04-14],[Outpatient],[Clinic referral],[Elective],[LG_Logan],[LG_Logan],[LG_US],[2015-04-14],[2015-04-15],[ADMINISTRATIVE_NOTE_OBSTYPE],[Consent Document]


In [16]:
(documents_concat.drop(columns=['DocServiceDate','EncounterRegistrationDate','LocationFacility','EncounterDischargeDate'],
                      inplace=True))

In [17]:
documents_concat['EncounterType'] = documents_concat['EncounterType'].str[0]
documents_concat['EncounterAdmissionSource'] = documents_concat['EncounterAdmissionSource'].str[0]
documents_concat['EncounterAdmissionType'] = documents_concat['EncounterAdmissionType'].str[0]
documents_concat['LocationBuilding'] = documents_concat['LocationBuilding'].str[0]
documents_concat['LocationNurseUnit'] = documents_concat['LocationNurseUnit'].str[0]

In [18]:
documents_concat.head()

,encounterId,EncounterType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit,DiscernDocumentType,StandardDocumentType
0,25833853,Outpatient,Clinic referral,Elective,LG_Med Art Bldg,LG_Budge Lab,[ADMINISTRATIVE_NOTE_OBSTYPE],[Consent Document]
1,25857613,Emergency,Information not available,Emergency,LG_Logan,LG_ED,"[CLINICAL_NOTE_OBSTYPE, ADMINISTRATIVE_NOTE_OBSTYPE]","[Consent Document, Emergency department Note, Emergency department Triage note]"
2,25858962,Emergency,Information not available,Emergency,LG_Logan,LG_ED,"[CLINICAL_NOTE_OBSTYPE, ADMINISTRATIVE_NOTE_OBSTYPE]","[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
3,25873399,Outpatient,Information not available,Unknown,LG_Logan,LG_Main Lab,[PATHOLOGY_REPORT_OBSTYPE],[Surgical pathology study]
4,26095725,Outpatient,Clinic referral,Elective,LG_Logan,LG_US,[ADMINISTRATIVE_NOTE_OBSTYPE],[Consent Document]


In [19]:
documents_concat.isnull().sum()

encounterId                    0
EncounterType                  0
EncounterAdmissionSource    6111
EncounterAdmissionType         0
LocationBuilding               0
LocationNurseUnit              0
DiscernDocumentType            0
StandardDocumentType           0
dtype: int64

In [20]:
documents_concat.describe()

,encounterId,EncounterType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit,DiscernDocumentType,StandardDocumentType
count,1000000,1000000,993889,1000000,1000000,1000000,1000000,1000000
unique,1000000,3,7,6,364,1500,71,3298
top,28719013,Outpatient,Clinic referral,Elective,MK_McKay-Dee,CL_ORPDS,[ADMINISTRATIVE_NOTE_OBSTYPE],[Consent Document]
freq,1,827305,715914,819225,89036,75634,581676,577331


In [21]:
documents_final = documents_concat.loc[documents_concat['encounterId'] != '0']

In [22]:
documents_final.shape

(1000000, 8)

In [23]:
documents_final = documents_concat.loc[documents_concat['EncounterAdmissionSource'].notnull()]

In [24]:
documents_final.shape

(993889, 8)

In [25]:
documents_final.head()

,encounterId,EncounterType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit,DiscernDocumentType,StandardDocumentType
0,25833853,Outpatient,Clinic referral,Elective,LG_Med Art Bldg,LG_Budge Lab,[ADMINISTRATIVE_NOTE_OBSTYPE],[Consent Document]
1,25857613,Emergency,Information not available,Emergency,LG_Logan,LG_ED,"[CLINICAL_NOTE_OBSTYPE, ADMINISTRATIVE_NOTE_OBSTYPE]","[Consent Document, Emergency department Note, Emergency department Triage note]"
2,25858962,Emergency,Information not available,Emergency,LG_Logan,LG_ED,"[CLINICAL_NOTE_OBSTYPE, ADMINISTRATIVE_NOTE_OBSTYPE]","[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
3,25873399,Outpatient,Information not available,Unknown,LG_Logan,LG_Main Lab,[PATHOLOGY_REPORT_OBSTYPE],[Surgical pathology study]
4,26095725,Outpatient,Clinic referral,Elective,LG_Logan,LG_US,[ADMINISTRATIVE_NOTE_OBSTYPE],[Consent Document]


In [26]:
documents_final_standard = documents_final[['encounterId', 'StandardDocumentType']]
documents_final_standard.shape

(993889, 2)

In [27]:
documents_final.isnull().sum()

encounterId                 0
EncounterType               0
EncounterAdmissionSource    0
EncounterAdmissionType      0
LocationBuilding            0
LocationNurseUnit           0
DiscernDocumentType         0
StandardDocumentType        0
dtype: int64

In [28]:
documents_final = documents_final.loc[documents_final['EncounterAdmissionSource'] != 'Information not available']

In [29]:
documents_final.shape

(993535, 8)

In [30]:
documents_final = documents_final.loc[documents_final['EncounterAdmissionType'] != 'Unknown']

In [31]:
documents_final.shape

(993535, 8)

In [32]:
documents_final['EncounterAdmissionType'].value_counts()

Elective                                  813183
Urgent                                    136704
Emergency                                  42719
Accident                                     917
Newborn (Birth in healthcare facility)        12
Name: EncounterAdmissionType, dtype: int64

In [33]:
documents_final['EncounterAdmissionSource'].value_counts()

Clinic referral                                     715914
Hospital admission from non-health care facility    274827
Court/law enforcement                                 1964
Transfer from another health care facility             401
Transfer from a hospital                               349
Transfer from a skilled nursing facility                80
Name: EncounterAdmissionSource, dtype: int64

In [34]:
documents_final['LocationBuilding'].value_counts()

MK_McKay-Dee          88971
CL_Outreach           74622
DX_Dixie              49866
AF_Amer Fork          45188
LG_Logan              43307
CC_Cedar City         33212
DX_Health Ctr         29393
LG_Med Art Bldg       28757
UV_UVOC               26001
CL_Q_U_Clnt           25230
IM_2 Eccles OP        23506
PC_Primary            23174
UV_Utah Valley        21409
RV_2 Riverton         21198
IM_9 ED               20245
SV_Sevier             19795
PK_Park City          17193
AV_Alta View          15145
CL_G_K                14181
IM_8 BreastCntr       14059
CA_Cassia             13940
LD_6 LDS              12934
UV_Pedersen Twr       12862
CL_Central Lab        12377
BR_Bear River         12291
PC_Eccles OP          11191
HV_Heber              10872
OR_Orem               10751
RV_Emergency           9622
DX_Hurricane           9452
DX_400 E Campus        8400
TO_TOSH                8205
CL_V_Z                 8106
UVFP_FamilyMed         7658
SP_Sanpete             7352
IM_4 Heart Lung     

In [35]:
documents_final.dtypes

encounterId                 object
EncounterType               object
EncounterAdmissionSource    object
EncounterAdmissionType      object
LocationBuilding            object
LocationNurseUnit           object
DiscernDocumentType         object
StandardDocumentType        object
dtype: object

In [36]:
documents_final['encounterId'] = documents_final['encounterId'].astype(str)
documents_final['EncounterType'] = documents_final['EncounterType'].astype(str)
documents_final['EncounterAdmissionSource'] = documents_final['EncounterAdmissionSource'].astype(str)
documents_final['EncounterAdmissionType'] = documents_final['EncounterAdmissionType'].astype(str)
documents_final['LocationBuilding'] = documents_final['LocationBuilding'].astype(str)
documents_final['LocationNurseUnit'] = documents_final['LocationNurseUnit'].astype(str)

In [37]:
documents_final.isnull().sum()

encounterId                 0
EncounterType               0
EncounterAdmissionSource    0
EncounterAdmissionType      0
LocationBuilding            0
LocationNurseUnit           0
DiscernDocumentType         0
StandardDocumentType        0
dtype: int64

In [38]:
documents_final.shape

(993535, 8)

In [39]:
documents_final.head()

,encounterId,EncounterType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit,DiscernDocumentType,StandardDocumentType
0,25833853,Outpatient,Clinic referral,Elective,LG_Med Art Bldg,LG_Budge Lab,[ADMINISTRATIVE_NOTE_OBSTYPE],[Consent Document]
4,26095725,Outpatient,Clinic referral,Elective,LG_Logan,LG_US,[ADMINISTRATIVE_NOTE_OBSTYPE],[Consent Document]
5,26141171,Outpatient,Clinic referral,Elective,LG_Logan,LG_CT,[ADMINISTRATIVE_NOTE_OBSTYPE],[Consent Document]
6,26159446,Outpatient,Clinic referral,Elective,LG_Logan,LG_Main Lab,[PATHOLOGY_REPORT_OBSTYPE],[Surgical pathology study]
7,26204744,Outpatient,Clinic referral,Elective,LG_Logan,LG_Mammo,[ADMINISTRATIVE_NOTE_OBSTYPE],[Consent Document]


In [40]:
documents_final_standard = ( documents_final[['encounterId', 
                                                'EncounterType', 
                                                'StandardDocumentType', 
                                                'EncounterAdmissionSource',
                                                'EncounterAdmissionType',
                                                'LocationBuilding',
                                                'LocationNurseUnit']])

In [41]:
documents_final_standard['encounterId'] = documents_final_standard['encounterId'].astype(str)
documents_final_standard['EncounterType'] = documents_final_standard['EncounterType'].astype(str)
documents_final_standard['EncounterAdmissionSource'] = documents_final_standard['EncounterAdmissionSource'].astype(str)
documents_final_standard['EncounterAdmissionType'] = documents_final_standard['EncounterAdmissionType'].astype(str)
documents_final_standard['LocationBuilding'] = documents_final_standard['LocationBuilding'].astype(str)
documents_final_standard['LocationNurseUnit'] = documents_final_standard['LocationNurseUnit'].astype(str)

In [42]:
documents_final_standard.head()

,encounterId,EncounterType,StandardDocumentType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit
0,25833853,Outpatient,[Consent Document],Clinic referral,Elective,LG_Med Art Bldg,LG_Budge Lab
4,26095725,Outpatient,[Consent Document],Clinic referral,Elective,LG_Logan,LG_US
5,26141171,Outpatient,[Consent Document],Clinic referral,Elective,LG_Logan,LG_CT
6,26159446,Outpatient,[Surgical pathology study],Clinic referral,Elective,LG_Logan,LG_Main Lab
7,26204744,Outpatient,[Consent Document],Clinic referral,Elective,LG_Logan,LG_Mammo


In [43]:
transactions = list(documents_final_standard['StandardDocumentType'])

In [44]:
encoder = TransactionEncoder().fit(transactions)

In [45]:
onehot = encoder.transform(transactions)

In [46]:
onehot = pd.DataFrame(onehot, columns = encoder.columns_)

In [47]:
onehot.shape

(993535, 213)

In [48]:
documents_final_standard.dtypes

encounterId                 object
EncounterType               object
StandardDocumentType        object
EncounterAdmissionSource    object
EncounterAdmissionType      object
LocationBuilding            object
LocationNurseUnit           object
dtype: object

In [49]:
documents_final_standard.head()

,encounterId,EncounterType,StandardDocumentType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit
0,25833853,Outpatient,[Consent Document],Clinic referral,Elective,LG_Med Art Bldg,LG_Budge Lab
4,26095725,Outpatient,[Consent Document],Clinic referral,Elective,LG_Logan,LG_US
5,26141171,Outpatient,[Consent Document],Clinic referral,Elective,LG_Logan,LG_CT
6,26159446,Outpatient,[Surgical pathology study],Clinic referral,Elective,LG_Logan,LG_Main Lab
7,26204744,Outpatient,[Consent Document],Clinic referral,Elective,LG_Logan,LG_Mammo


In [50]:
onehot.head()

,Advance directives,Allergy and Immunology Consult note,Allergy and Immunology Outpatient Note,Allergy and Immunology Progress note,Allergy and Immunology procedure note,Anesthesiology Consult note,Anesthesiology Note,Anesthesiology Postoperative evaluation and management note,Anesthesiology Preoperative evaluation and management note,Anesthesiology procedure note,...,Urgent care center Note,Urology Consult note,Urology Outpatient Note,Urology Outpatient procedure note,Urology Progress note,Vascular surgery Consult note,Vascular surgery Outpatient Note,Vascular surgery procedure note,Well child visit note,Wound care management Note
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [51]:
encoder.columns_

['Advance directives',
 'Allergy and Immunology Consult note',
 'Allergy and Immunology Outpatient Note',
 'Allergy and Immunology Progress note',
 'Allergy and Immunology procedure note',
 'Anesthesiology Consult note',
 'Anesthesiology Note',
 'Anesthesiology Postoperative evaluation and management note',
 'Anesthesiology Preoperative evaluation and management note',
 'Anesthesiology procedure note',
 'Antepartum summary note',
 'Attending Progress note',
 'Audiology Note',
 'Audiology Progress note',
 'Blood banking and transfusion medicine Note',
 'Cardiac catheterization study',
 'Cardiac surgery Consult note',
 'Cardiology Consult note',
 'Cardiology Hospital Progress note',
 'Cardiology Outpatient Note',
 'Cardiology procedure note',
 'Care management note',
 'Clinical genetics Consult note',
 'Clinical genetics Outpatient Note',
 'Clinical pathology Consult note',
 'Colonoscopy study',
 'Consent Document',
 'Consult note',
 'Cytology report of Cervical or vaginal smear or scrap

In [52]:
documents_final_standard['StandardDocumentType'].explode().value_counts()

Consent Document                                                       861471
Emergency department Triage note                                       170937
Physician Emergency department Note                                    168539
Nurse Emergency department Note                                        146913
History and physical note                                               52323
Cytology report of Cervical or vaginal smear or scraping Cyto stain     43684
Surgical pathology study                                                40822
Anesthesiology Note                                                     31771
Surgical operation note                                                 26633
Outpatient Education note                                               26250
Family medicine Progress note                                           20677
Nurse Outpatient Note                                                   19746
Cardiology procedure note                                       

In [53]:
documents_final_standard.isnull().sum()

encounterId                 0
EncounterType               0
StandardDocumentType        0
EncounterAdmissionSource    0
EncounterAdmissionType      0
LocationBuilding            0
LocationNurseUnit           0
dtype: int64

In [54]:
documents_final_standard.dtypes

encounterId                 object
EncounterType               object
StandardDocumentType        object
EncounterAdmissionSource    object
EncounterAdmissionType      object
LocationBuilding            object
LocationNurseUnit           object
dtype: object

In [55]:
documents_final_standard['StandardDocumentType'].value_counts()

[Consent Document]                                                                                                                     575893
[Physician Emergency department Note, Consent Document, Nurse Emergency department Note, Emergency department Triage note]             139435
[Cytology report of Cervical or vaginal smear or scraping Cyto stain]                                                                   43435
[Surgical pathology study]                                                                                                              37121
[History and physical note, Anesthesiology Note, Consent Document, Surgical operation note]                                             24682
[Physician Emergency department Note, Consent Document, Emergency department Triage note]                                               15709
[Consent Document, Family medicine Progress note]                                                                                        9878
[Conse

In [56]:
documents_final_standard.reset_index(drop=True, inplace=True)

In [57]:
top30 = documents_final_standard['StandardDocumentType'].explode().value_counts().nlargest(30)

In [58]:
documents_final_standard['StandardDocumentType'].explode().value_counts().shape[0]

213

In [59]:
top30.index.tolist()

['Consent Document',
 'Emergency department Triage note',
 'Physician Emergency department Note',
 'Nurse Emergency department Note',
 'History and physical note',
 'Cytology report of Cervical or vaginal smear or scraping Cyto stain',
 'Surgical pathology study',
 'Anesthesiology Note',
 'Surgical operation note',
 'Outpatient Education note',
 'Family medicine Progress note',
 'Nurse Outpatient Note',
 'Cardiology procedure note',
 'Maternal and fetal medicine Note',
 'Discharge instructions',
 'Procedure note',
 'Sleep medicine Outpatient Note',
 'Endocrinology Note',
 'Oncology Progress note',
 'Cardiology Outpatient Note',
 'Internal medicine Outpatient Note',
 'Care management note',
 'Audiology Note',
 'Non-gynecological cytology method study',
 'Pulmonary Outpatient Note',
 'Endocrinology Outpatient Note',
 'Obstetrics Progress note',
 'Gynecology Outpatient Note',
 'MG Breast Views',
 'Orthopaedic surgery Outpatient Note']

In [60]:
'|'.join(top30.index.tolist())

'Consent Document|Emergency department Triage note|Physician Emergency department Note|Nurse Emergency department Note|History and physical note|Cytology report of Cervical or vaginal smear or scraping Cyto stain|Surgical pathology study|Anesthesiology Note|Surgical operation note|Outpatient Education note|Family medicine Progress note|Nurse Outpatient Note|Cardiology procedure note|Maternal and fetal medicine Note|Discharge instructions|Procedure note|Sleep medicine Outpatient Note|Endocrinology Note|Oncology Progress note|Cardiology Outpatient Note|Internal medicine Outpatient Note|Care management note|Audiology Note|Non-gynecological cytology method study|Pulmonary Outpatient Note|Endocrinology Outpatient Note|Obstetrics Progress note|Gynecology Outpatient Note|MG Breast Views|Orthopaedic surgery Outpatient Note'

In [193]:
top30.index

Index(['Consent Document', 'Emergency department Triage note',
       'Physician Emergency department Note',
       'Nurse Emergency department Note', 'History and physical note',
       'Cytology report of Cervical or vaginal smear or scraping Cyto stain',
       'Surgical pathology study', 'Anesthesiology Note',
       'Surgical operation note', 'Outpatient Education note',
       'Family medicine Progress note', 'Nurse Outpatient Note',
       'Cardiology procedure note', 'Maternal and fetal medicine Note',
       'Discharge instructions', 'Procedure note',
       'Sleep medicine Outpatient Note', 'Endocrinology Note',
       'Oncology Progress note', 'Cardiology Outpatient Note',
       'Internal medicine Outpatient Note', 'Care management note',
       'Audiology Note', 'Non-gynecological cytology method study',
       'Pulmonary Outpatient Note', 'Endocrinology Outpatient Note',
       'Obstetrics Progress note', 'Gynecology Outpatient Note',
       'MG Breast Views', 'Orthopaedi

In [194]:
documents_final_standard['StandardDocumentType'].dtype

dtype('O')

In [195]:
# document_updated_oupatient_standard_largest = ( documents_final_standard[
#                                         documents_final_standard['StandardDocumentType']
#                                             .str.join(',').str.contains
#                                             ('|'.join(
#                                                 top20.index.tolist()
#                                             ))])

In [196]:
 document_updated_oupatient_standard_largest = ( documents_final_standard[
             documents_final_standard['StandardDocumentType']
     .apply(lambda lst: all(x in top30.index.tolist() for x in lst))])

In [197]:
document_updated_oupatient_standard_largest.shape

(988633, 7)

In [198]:
document_updated_oupatient_standard_largest['StandardDocumentType'].explode().value_counts().shape[0]

209

In [67]:
document_updated_oupatient_standard_largest.describe()

,encounterId,EncounterType,StandardDocumentType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit
count,964187,964187,964187,964187,964187,964187,964187
unique,964187,3,726,6,5,263,1331
top,28719013,Outpatient,[Consent Document],Clinic referral,Elective,MK_McKay-Dee,CL_ORPDS
freq,1,795491,575893,696772,788990,84345,74076


In [68]:
document_updated_oupatient_standard_largest['EncounterAdmissionSource'].unique()

array(['Clinic referral',
       'Hospital admission from non-health care facility',
       'Court/law enforcement',
       'Transfer from another health care facility',
       'Transfer from a hospital',
       'Transfer from a skilled nursing facility'], dtype=object)

In [69]:
document_updated_oupatient_standard_largest['EncounterAdmissionType'].unique()

array(['Elective', 'Emergency', 'Urgent', 'Accident',
       'Newborn (Birth in healthcare facility)'], dtype=object)

In [70]:
document_updated_oupatient_standard_largest.shape

(964187, 7)

In [71]:
document_updated_oupatient_standard_largest.describe(include='all')

,encounterId,EncounterType,StandardDocumentType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit
count,964187,964187,964187,964187,964187,964187,964187
unique,964187,3,726,6,5,263,1331
top,28719013,Outpatient,[Consent Document],Clinic referral,Elective,MK_McKay-Dee,CL_ORPDS
freq,1,795491,575893,696772,788990,84345,74076


In [72]:
document_updated_oupatient_standard_largest.reset_index(drop=True, inplace=True)

In [73]:
top100_multiple = document_updated_oupatient_standard_largest['StandardDocumentType'].value_counts().nlargest(100)

In [74]:
 document_updated_oupatient_standard_multiple = ( document_updated_oupatient_standard_largest[
             document_updated_oupatient_standard_largest['StandardDocumentType']
     .apply(lambda x:  any(x == y for y in top100_multiple.index.tolist()))])

In [75]:
document_updated_oupatient_standard_multiple.shape

(954515, 7)

In [76]:
document_updated_oupatient_standard_multiple['StandardDocumentType'].value_counts()

[Consent Document]                                                                                                                     575893
[Physician Emergency department Note, Consent Document, Nurse Emergency department Note, Emergency department Triage note]             139435
[Cytology report of Cervical or vaginal smear or scraping Cyto stain]                                                                   43435
[Surgical pathology study]                                                                                                              37121
[History and physical note, Anesthesiology Note, Consent Document, Surgical operation note]                                             24682
[Physician Emergency department Note, Consent Document, Emergency department Triage note]                                               15709
[Consent Document, Family medicine Progress note]                                                                                        9878
[Conse

In [77]:
document_updated_oupatient_standard_multiple['StandardDocumentType'].explode().value_counts()

Consent Document                                                       835744
Emergency department Triage note                                       166219
Physician Emergency department Note                                    163619
Nurse Emergency department Note                                        146003
History and physical note                                               45646
Cytology report of Cervical or vaginal smear or scraping Cyto stain     43435
Surgical pathology study                                                39800
Anesthesiology Note                                                     29803
Surgical operation note                                                 25624
Outpatient Education note                                               20634
Family medicine Progress note                                           19209
Cardiology procedure note                                               16606
Nurse Outpatient Note                                           

In [78]:
document_updated_oupatient_standard_multiple.reset_index(drop=True, inplace=True)

In [79]:
document_updated_oupatient_standard_multiple.shape

(954515, 7)

In [80]:
document_updated_oupatient_standard_multiple.head()

,encounterId,EncounterType,StandardDocumentType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit
0,25833853,Outpatient,[Consent Document],Clinic referral,Elective,LG_Med Art Bldg,LG_Budge Lab
1,26095725,Outpatient,[Consent Document],Clinic referral,Elective,LG_Logan,LG_US
2,26141171,Outpatient,[Consent Document],Clinic referral,Elective,LG_Logan,LG_CT
3,26159446,Outpatient,[Surgical pathology study],Clinic referral,Elective,LG_Logan,LG_Main Lab
4,26204744,Outpatient,[Consent Document],Clinic referral,Elective,LG_Logan,LG_Mammo


In [81]:
document_updated_oupatient_standard_multiple.describe()

,encounterId,EncounterType,StandardDocumentType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit
count,954515,954515,954515,954515,954515,954515,954515
unique,954515,3,100,6,5,243,1300
top,28719013,Outpatient,[Consent Document],Clinic referral,Elective,MK_McKay-Dee,CL_ORPDS
freq,1,786967,575893,690082,780756,83921,74019


In [82]:
(document_updated_oupatient_standard_multiple.
to_pickle("document_updated_outpatient_standard_multiple_data_analysis.pkl.compress", compression="gzip", protocol=4))

In [5]:
document_updated_outpatient_standard_multiple_data_analysis = (pd.read_pickle
                                        ("document_updated_outpatient_standard_multiple_data_analysis.pkl.compress", compression="gzip"))

In [6]:
document_updated_outpatient_standard_multiple_data_analysis.head()

,encounterId,EncounterType,StandardDocumentType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit
0,25833853,Outpatient,[Consent Document],Clinic referral,Elective,LG_Med Art Bldg,LG_Budge Lab
1,26095725,Outpatient,[Consent Document],Clinic referral,Elective,LG_Logan,LG_US
2,26141171,Outpatient,[Consent Document],Clinic referral,Elective,LG_Logan,LG_CT
3,26159446,Outpatient,[Surgical pathology study],Clinic referral,Elective,LG_Logan,LG_Main Lab
4,26204744,Outpatient,[Consent Document],Clinic referral,Elective,LG_Logan,LG_Mammo


In [7]:
targetName = 'StandardDocumentType'
targetSeries = document_updated_outpatient_standard_multiple_data_analysis[targetName]
del document_updated_outpatient_standard_multiple_data_analysis[targetName]
document_updated_outpatient_standard_multiple_data_analysis.insert(6, targetName, targetSeries)

In [8]:
document_updated_outpatient_standard_multiple_data_analysis.shape

(954515, 7)

In [9]:
document_updated_outpatient_unique = (document_updated_outpatient_standard_multiple_data_analysis
                                               [document_updated_outpatient_standard_multiple_data_analysis
                                                ['StandardDocumentType'].str.len() == 1])


In [10]:
document_updated_outpatient_unique.head(20)

,encounterId,EncounterType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit,StandardDocumentType
0,25833853,Outpatient,Clinic referral,Elective,LG_Med Art Bldg,LG_Budge Lab,[Consent Document]
1,26095725,Outpatient,Clinic referral,Elective,LG_Logan,LG_US,[Consent Document]
2,26141171,Outpatient,Clinic referral,Elective,LG_Logan,LG_CT,[Consent Document]
3,26159446,Outpatient,Clinic referral,Elective,LG_Logan,LG_Main Lab,[Surgical pathology study]
4,26204744,Outpatient,Clinic referral,Elective,LG_Logan,LG_Mammo,[Consent Document]
5,26214786,Outpatient,Clinic referral,Elective,LG_Logan,LG_Mammo,[Consent Document]
7,26242393,Outpatient,Clinic referral,Elective,LG_Med Art Bldg,LG_Budge Lab,[Consent Document]
8,26265757,Outpatient,Clinic referral,Elective,LG_Med Art Bldg,LG_Budge Lab,[Consent Document]
10,26298858,Outpatient,Clinic referral,Elective,LG_W & NB Cntr,LG_WC Lab,[Consent Document]
11,26303183,Outpatient,Clinic referral,Elective,LG_Logan,LG_Cardio,[Consent Document]


In [11]:
consent = ['Consent Document']

In [21]:
document_updated_outpatient_standard_unique_consent = ( document_updated_outpatient_unique[
             document_updated_outpatient_unique['StandardDocumentType']
     .apply(lambda lst: any(x in consent for x in lst))])

In [22]:
document_updated_outpatient_standard_unique_consent.head()

,encounterId,EncounterType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit,StandardDocumentType
0,25833853,Outpatient,Clinic referral,Elective,LG_Med Art Bldg,LG_Budge Lab,[Consent Document]
1,26095725,Outpatient,Clinic referral,Elective,LG_Logan,LG_US,[Consent Document]
2,26141171,Outpatient,Clinic referral,Elective,LG_Logan,LG_CT,[Consent Document]
4,26204744,Outpatient,Clinic referral,Elective,LG_Logan,LG_Mammo,[Consent Document]
5,26214786,Outpatient,Clinic referral,Elective,LG_Logan,LG_Mammo,[Consent Document]


In [23]:
document_updated_outpatient_standard_unique_consent.shape

(575893, 7)

In [24]:
document_updated_outpatient_standard_unique_consent['StandardDocumentType'].explode().value_counts()

Consent Document    575893
Name: StandardDocumentType, dtype: int64

In [25]:
document_updated_outpatient_standard_unique_consent.shape

(575893, 7)

In [26]:
document_updated_outpatient_standard_unique_consent = (
                    document_updated_outpatient_standard_unique_consent.drop_duplicates
                        (subset=['EncounterType','EncounterAdmissionSource', 'EncounterAdmissionType',
                                 'LocationBuilding', 'LocationNurseUnit']))

In [27]:
document_updated_outpatient_standard_unique_consent.shape

(2394, 7)

In [28]:
document_updated_outpatient_standard_unique_consent.tail(20)

,encounterId,EncounterType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit,StandardDocumentType
930071,32273153,Outpatient,Hospital admission from non-health care facility,Urgent,SV_Sevier,SV_XRAY,[Consent Document]
930585,43541323,Outpatient,Hospital admission from non-health care facility,Urgent,LG_Logan,LG_Main Lab,[Consent Document]
931264,57878685,Outpatient,Clinic referral,Urgent,AF_Amer Fork,AF_EKG,[Consent Document]
931630,64736843,Outpatient,Hospital admission from non-health care facility,Emergency,PK_Park City,PK_MRI,[Consent Document]
931936,71567099,Outpatient,Clinic referral,Emergency,PC_Primary,PC_MRI,[Consent Document]
932177,76141208,Outpatient,Clinic referral,Emergency,AF_Amer Fork,AF_EKG,[Consent Document]
935881,90536471,Outpatient,Hospital admission from non-health care facility,Urgent,LH_Layton,LH_US,[Consent Document]
937248,55058350,Outpatient,Hospital admission from non-health care facility,Emergency,AF_Amer Fork,AF_DEXA,[Consent Document]
938586,80346790,Outpatient,Clinic referral,Elective,CL_G_K,CL_HLCL,[Consent Document]
939532,38010164,Preadmit,Clinic referral,Elective,MK_McKay-Dee,MK_US,[Consent Document]


In [31]:
audiology = ['Audiology Note']

In [32]:
document_updated_outpatient_standard_unique_audiology = ( document_updated_outpatient_unique[
             document_updated_outpatient_unique['StandardDocumentType']
     .apply(lambda lst: any(x in audiology for x in lst))])

In [34]:
document_updated_outpatient_standard_unique_audiology.shape

(1072, 7)

In [35]:
document_updated_outpatient_standard_unique_audiology = (
                    document_updated_outpatient_standard_unique_audiology.drop_duplicates
                        (subset=['EncounterType','EncounterAdmissionSource', 'EncounterAdmissionType',
                                 'LocationBuilding', 'LocationNurseUnit']))

In [38]:
document_updated_outpatient_standard_unique_audiology.tail(20)

,encounterId,EncounterType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit,StandardDocumentType
127147,64558306,Outpatient,Hospital admission from non-health care facility,Elective,MK_McKay-Dee,MK_Peds_Aud,[Audiology Note]
133000,57826749,Outpatient,Clinic referral,Elective,SP_Sanpete,SP_Lab,[Audiology Note]
147953,48987781,Outpatient,Clinic referral,Elective,PC_Layton Rehab,PC_LA_Audiology,[Audiology Note]
157300,50224890,Outpatient,Clinic referral,Elective,TO_Midtown,TO_MP_PT,[Audiology Note]
178643,43320743,Outpatient,Clinic referral,Elective,CL_G_K,CL_SVHCT,[Audiology Note]
196448,29494327,Outpatient,Clinic referral,Elective,CL_A_F_Clnt,CL_CUMCL_Clnt,[Audiology Note]
223217,81137293,Outpatient,Hospital admission from non-health care facility,Elective,PC_Tville Rehab,PC_TA_Audiology,[Audiology Note]
257302,26738043,Outpatient,Clinic referral,Elective,BR_Bear River,BR_Audiology,[Audiology Note]
269469,26663742,Outpatient,Hospital admission from non-health care facility,Elective,BDSP_ENT,BDSP_ENT,[Audiology Note]
281899,28906230,Outpatient,Clinic referral,Elective,LG_Logan,LG_Spine_Aud,[Audiology Note]


In [39]:
document_updated_outpatient_multiple = (document_updated_outpatient_standard_multiple_data_analysis
 [document_updated_outpatient_standard_multiple_data_analysis['StandardDocumentType'].str.len() > 1])

In [40]:
document_updated_outpatient_multiple.shape

(279373, 7)

In [41]:
document_updated_outpatient_multiple.head(10)

,encounterId,EncounterType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit,StandardDocumentType
6,26234371,Emergency,Hospital admission from non-health care facility,Emergency,LG_Logan,LG_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
9,26295700,Emergency,Hospital admission from non-health care facility,Emergency,LG_Logan,LG_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
14,26445800,Emergency,Hospital admission from non-health care facility,Emergency,LG_Logan,LG_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
15,26469886,Emergency,Hospital admission from non-health care facility,Emergency,LG_Logan,LG_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
16,26539767,Emergency,Hospital admission from non-health care facility,Urgent,LG_Logan,LG_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
17,26587822,Emergency,Hospital admission from non-health care facility,Urgent,BR_Bear River,BR_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
18,26643282,Emergency,Hospital admission from non-health care facility,Urgent,LG_Logan,LG_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
20,26789349,Emergency,Hospital admission from non-health care facility,Urgent,LG_Logan,LG_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
24,27298847,Outpatient,Hospital admission from non-health care facility,Elective,CRIM_IM,CRIM_IM,"[Consent Document, Internal medicine Outpatient Note]"
30,27414892,Emergency,Hospital admission from non-health care facility,Urgent,MK_McKay-Dee,MK_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"


In [42]:
document_updated_outpatient_consent_multiple = ( document_updated_outpatient_multiple[
             document_updated_outpatient_multiple['StandardDocumentType']
     .apply(lambda lst: any(x in consent for x in lst))])

In [43]:
document_updated_outpatient_consent_multiple.head(20)

,encounterId,EncounterType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit,StandardDocumentType
6,26234371,Emergency,Hospital admission from non-health care facility,Emergency,LG_Logan,LG_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
9,26295700,Emergency,Hospital admission from non-health care facility,Emergency,LG_Logan,LG_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
14,26445800,Emergency,Hospital admission from non-health care facility,Emergency,LG_Logan,LG_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
15,26469886,Emergency,Hospital admission from non-health care facility,Emergency,LG_Logan,LG_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
16,26539767,Emergency,Hospital admission from non-health care facility,Urgent,LG_Logan,LG_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
17,26587822,Emergency,Hospital admission from non-health care facility,Urgent,BR_Bear River,BR_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
18,26643282,Emergency,Hospital admission from non-health care facility,Urgent,LG_Logan,LG_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
20,26789349,Emergency,Hospital admission from non-health care facility,Urgent,LG_Logan,LG_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
24,27298847,Outpatient,Hospital admission from non-health care facility,Elective,CRIM_IM,CRIM_IM,"[Consent Document, Internal medicine Outpatient Note]"
30,27414892,Emergency,Hospital admission from non-health care facility,Urgent,MK_McKay-Dee,MK_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"


In [44]:
document_updated_outpatient_consent_multiple = (
                    document_updated_outpatient_consent_multiple.drop_duplicates
                        (subset=['EncounterType','EncounterAdmissionSource', 'EncounterAdmissionType',
                                 'LocationBuilding', 'LocationNurseUnit']))

In [45]:
document_updated_outpatient_consent_multiple.shape

(1859, 7)

In [46]:
document_updated_outpatient_consent_multiple.head(20)

,encounterId,EncounterType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit,StandardDocumentType
6,26234371,Emergency,Hospital admission from non-health care facility,Emergency,LG_Logan,LG_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
16,26539767,Emergency,Hospital admission from non-health care facility,Urgent,LG_Logan,LG_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
17,26587822,Emergency,Hospital admission from non-health care facility,Urgent,BR_Bear River,BR_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
24,27298847,Outpatient,Hospital admission from non-health care facility,Elective,CRIM_IM,CRIM_IM,"[Consent Document, Internal medicine Outpatient Note]"
30,27414892,Emergency,Hospital admission from non-health care facility,Urgent,MK_McKay-Dee,MK_ED,"[Physician Emergency department Note, Consent Document, Emergency department Triage note]"
42,27605662,Outpatient,Hospital admission from non-health care facility,Elective,CA_Cassia,CA_LD,"[Obstetrics Progress note, Consent Document]"
51,27703846,Outpatient,Hospital admission from non-health care facility,Elective,CRSC_ObGyn,CRSC_ObGyn,"[Gynecology Outpatient Note, Consent Document]"
53,27742644,Outpatient,Clinic referral,Elective,MK_McKay-Dee,MK_Walkin Lab,"[Consent Document, Cardiology procedure note]"
63,28234107,Outpatient,Clinic referral,Elective,CRFO_FamilyMed,CRFO_FamilyMed,"[Consent Document, Outpatient Education note]"
69,28430351,Outpatient,Clinic referral,Elective,CROR_Ortho,CROR_Ortho,"[Consent Document, Orthopaedic surgery Outpatient Note]"


In [47]:
anesthilogy = ['Anesthesiology Note']

In [48]:
document_updated_outpatient_standard_multiple_data_analysis.shape

(954515, 7)

In [49]:
document_updated_outpatient_anesthesiology_multiple = ( document_updated_outpatient_multiple[
             document_updated_outpatient_multiple['StandardDocumentType']
     .apply(lambda lst: any(x in anesthilogy for x in lst))])

In [50]:
document_updated_outpatient_anesthesiology_multiple.shape

(29803, 7)

In [51]:
document_updated_outpatient_anesthesiology_multiple.head()

,encounterId,EncounterType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit,StandardDocumentType
336,35386606,Outpatient,Clinic referral,Elective,UV_East Tower,UV_Med Surg,"[History and physical note, Anesthesiology Note, Consent Document, Surgical operation note]"
350,35614865,Outpatient,Clinic referral,Elective,UV_UVOC,UV_OC_SS_SDS,"[History and physical note, Anesthesiology Note, Consent Document, Surgical operation note]"
363,35767045,Outpatient,Clinic referral,Elective,LG_Med Art Bldg,LG_SS_SDS,"[History and physical note, Anesthesiology Note, Consent Document, Surgical operation note]"
434,37214220,Outpatient,Hospital admission from non-health care facility,Elective,MS_Surgical Ctr,MS_SS_SDS,"[History and physical note, Anesthesiology Note, Consent Document, Surgical operation note]"
461,37741122,Outpatient,Clinic referral,Elective,UV_East Tower,UV_SS_SDS,"[History and physical note, Anesthesiology Note, Consent Document, Surgical operation note]"


In [52]:
document_updated_outpatient_anesthesiology_multiple = (
                    document_updated_outpatient_anesthesiology_multiple.drop_duplicates
                        (subset=['EncounterType','EncounterAdmissionSource', 'EncounterAdmissionType',
                                 'LocationBuilding', 'LocationNurseUnit']))

In [53]:
document_updated_outpatient_anesthesiology_multiple.shape

(451, 7)

In [54]:
document_updated_outpatient_anesthesiology_multiple.head(20)

,encounterId,EncounterType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit,StandardDocumentType
336,35386606,Outpatient,Clinic referral,Elective,UV_East Tower,UV_Med Surg,"[History and physical note, Anesthesiology Note, Consent Document, Surgical operation note]"
350,35614865,Outpatient,Clinic referral,Elective,UV_UVOC,UV_OC_SS_SDS,"[History and physical note, Anesthesiology Note, Consent Document, Surgical operation note]"
363,35767045,Outpatient,Clinic referral,Elective,LG_Med Art Bldg,LG_SS_SDS,"[History and physical note, Anesthesiology Note, Consent Document, Surgical operation note]"
434,37214220,Outpatient,Hospital admission from non-health care facility,Elective,MS_Surgical Ctr,MS_SS_SDS,"[History and physical note, Anesthesiology Note, Consent Document, Surgical operation note]"
461,37741122,Outpatient,Clinic referral,Elective,UV_East Tower,UV_SS_SDS,"[History and physical note, Anesthesiology Note, Consent Document, Surgical operation note]"
482,38358403,Outpatient,Hospital admission from non-health care facility,Elective,CC_Cedar City,CC_SS_Endo,"[History and physical note, Anesthesiology Note, Consent Document, Procedure note]"
537,39481399,Outpatient,Clinic referral,Elective,AF_Amer Fork,AF_SS_PACU,"[History and physical note, Anesthesiology Note, Consent Document, Surgical operation note]"
598,40555026,Outpatient,Clinic referral,Elective,AF_Amer Fork,AF_SS_SDS,"[History and physical note, Anesthesiology Note, Consent Document, Surgical operation note]"
759,44036029,Outpatient,Clinic referral,Elective,MS_Surgical Ctr,MS_SS_SDS,"[History and physical note, Anesthesiology Note, Consent Document, Surgical operation note]"
842,45906364,Outpatient,Hospital admission from non-health care facility,Elective,IM_5 PatientTwr,IM_SS_SDS Twr,"[History and physical note, Anesthesiology Note, Consent Document, Surgical operation note]"


In [55]:
audilogy = ['Audiology Note']

In [56]:
document_updated_outpatient_audilogy_multiple = ( document_updated_outpatient_multiple[
             document_updated_outpatient_multiple['StandardDocumentType']
     .apply(lambda lst: any(x in audilogy for x in lst))])

In [57]:
document_updated_outpatient_audilogy_multiple = (
                    document_updated_outpatient_audilogy_multiple.drop_duplicates
                        (subset=['EncounterType','EncounterAdmissionSource', 'EncounterAdmissionType',
                                 'LocationBuilding', 'LocationNurseUnit']))

In [58]:
document_updated_outpatient_audilogy_multiple.head(20)

,encounterId,EncounterType,EncounterAdmissionSource,EncounterAdmissionType,LocationBuilding,LocationNurseUnit,StandardDocumentType
2595,80737446,Outpatient,Clinic referral,Elective,PC_Tville Rehab,PC_TA_Audiology,"[Consent Document, Audiology Note]"
2917,88184247,Outpatient,Hospital admission from non-health care facility,Elective,AFHA_Audiology,AFHA_Audiology,"[Consent Document, Audiology Note]"
4771,64581562,Outpatient,Clinic referral,Elective,UV_Sorenson Twr,UV_ST_Audiology,"[Consent Document, Audiology Note]"
4894,66999865,Outpatient,Clinic referral,Elective,PC_Rvrtn Rehab,PC_RT_Audiology,"[Consent Document, Audiology Note]"
7203,51675784,Outpatient,Clinic referral,Elective,AFHA_Audiology,AFHA_Audiology,"[Consent Document, Audiology Note]"
9031,88033745,Outpatient,Clinic referral,Elective,UV_Pedersen Twr,UV_DEXA,"[Consent Document, Audiology Note]"
12878,44344874,Outpatient,Clinic referral,Elective,IM_1 SOT,IM_Audiology,"[Consent Document, Audiology Note]"
13166,49796279,Outpatient,Clinic referral,Elective,TO_Midtown,TO_MP_Aud,"[Consent Document, Audiology Note]"
14413,73888319,Outpatient,Clinic referral,Elective,MK_McKay-Dee,MK_Peds_Aud,"[Consent Document, Audiology Note]"
14994,85386694,Outpatient,Clinic referral,Elective,PC_Primary,PC_Audiology,"[Consent Document, Audiology Note]"
